# Zadanie 4 (7 pkt)
Celem zadania jest zaimplementowanie algorytmu drzewa decyzyjnego ID3 dla zadania klasyfikacji. Trening i test należy przeprowadzić dla zbioru Iris. Proszę przeprowadzić eksperymenty najpierw dla DOKŁADNIE takiego podziału zbioru testowego i treningowego jak umieszczony poniżej. W dalszej części należy przeprowadzić analizę działania drzewa dla różnych wartości parametrów. Proszę korzystać z przygotowanego szkieletu programu, oczywiście można go modyfikować według potrzeb. Wszelkie elementy szkieletu zostaną wyjaśnione na zajęciach.

* Implementacja funkcji entropii - **0.5 pkt**
* Implementacja funkcji entropii zbioru - **0.5 pkt**
* Implementacja funkcji information gain - **0.5 pkt**
* Zbudowanie poprawnie działającego drzewa klasyfikacyjnego i przetestowanie go na wspomnianym wcześniej zbiorze testowym. Jeśli w liściu występuje kilka różnych klas, decyzją jest klasa większościowa. Policzenie accuracy i wypisanie parami klasy rzeczywistej i predykcji. - **4 pkt**
* Przeprowadzenie eksperymentów dla różnych głębokości drzew i podziałów zbioru treningowego i testowego (zmiana wartości argumentu test_size oraz usunięcie random_state). W tym przypadku dla każdego eksperymentu należy wykonać kilka uruchomień programu i wypisać dla każdego uruchomienia accuracy. - **1.5 pkt**


In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
import math
from collections import Counter
import numpy as np
from sympy.stats.rv import probability
from typing import Tuple

iris = load_iris()

x = iris.data
y = iris.target

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=123)

In [ ]:
def entropy_func(class_count: int, num_samples: int) -> float:
    probability = class_count / num_samples
    return -probability * np.log2(probability)


class Group:
    def __init__(self, group_classes):
        self.group_classes = group_classes
        self.entropy = self.group_entropy()

    def __len__(self) -> int:
        return self.group_classes.size

    def group_entropy(self) -> float:
        _, class_counts = np.unique(self.group_classes, return_counts=True)
        num_samples = len(self.group_classes)
        return sum(entropy_func(count, num_samples) for count in class_counts)


class Node:
    def __init__(self, split_feature=None, split_val=None, depth=None, child_node_a=None, child_node_b=None, val=None):
        self.split_feature = split_feature
        self.split_val = split_val
        self.depth = depth
        self.child_node_a = child_node_a
        self.child_node_b = child_node_b
        self.val = val

    def predict(self, data: np.ndarray) -> int:
        if self.val is not None:
            return self.val
        elif data[self.split_feature] >= self.split_val:
            return self.child_node_a.predict(data)
        else:
            return self.child_node_b.predict(data)


class DecisionTreeClassifier(object):
    def __init__(self, max_depth):
        self.depth = 0
        self.max_depth = max_depth
        self.tree = None

    @staticmethod
    def get_split_entropy(group_a: Group, group_b: Group) -> float:
        total_samples = len(group_a) + len(group_b)
        return (len(group_a) / total_samples) * group_a.group_entropy() + (len(group_b) / total_samples) * group_b.group_entropy()

    def get_information_gain(self, parent_group: Group, child_group_a: Group, child_group_b: Group) -> float:
        return parent_group.group_entropy() - self.get_split_entropy(child_group_a, child_group_b)

    def get_best_feature_split(self, feature_values: np.ndarray, classes: np.ndarray) -> Tuple[int, float]:
        parent_group = Group(classes)
        best_split_value = None
        best_gain = -1
        for split_feature in np.unique(feature_values):
            a_indices = feature_values <= split_feature
            b_indices = feature_values > split_feature
            if not np.any(a_indices) or not np.any(b_indices):
                continue

            group_a = Group(classes[a_indices])
            group_b = Group(classes[b_indices])
            gain = self.get_information_gain(parent_group, group_a, group_b)
            if gain > best_gain:
                best_split_value = split_feature
                best_gain = gain

        return best_split_value, best_gain

    def get_best_split(self, data: np.ndarray, classes: np.ndarray) -> Tuple[int, int, float]:
        best_split_value = None
        best_split_feature = None
        best_gain = -1

        for feature_index in range(data.shape[1]):
            feature_values = data[:, feature_index]
            split_value, gain = self.get_best_feature_split(feature_values, classes)
            if gain > best_gain:
                best_split_value = split_value
                best_split_feature = feature_index
                best_gain = gain

        return best_split_feature, best_split_value, best_gain

    def build_tree(self, data: np.ndarray, classes: np.ndarray, depth=0) -> Node:
        if depth >= self.max_depth or len(np.unique(classes)) == 1:
            majority_class = Counter(classes).most_common(1)[0][0]
            return Node(val=majority_class)

        split_feature, split_value, best_gain = self.get_best_split(data, classes)
        if split_feature is None:
            majority_class = Counter(classes).most_common(1)[0][0]
            return Node(val=majority_class)

        a_indices = data[:, split_feature] <= split_value
        b_indices = data[:, split_feature] > split_value
        child_node_a = self.build_tree(data[a_indices], classes[a_indices], depth + 1)
        child_node_b = self.build_tree(data[b_indices], classes[b_indices], depth + 1)

        return Node(split_feature, split_value, depth, child_node_a, child_node_b)

    def fit(self, data: np.ndarray, classes: np.ndarray):
        self.tree = self.build_tree(data, classes)

    def predict(self, data: np.ndarray) -> int:
        if self.tree is not None:
            return self.tree.predict(data)

In [ ]:
dc = DecisionTreeClassifier(3)
dc.build_tree(x_train, y_train)
for sample, gt in zip(x_test, y_test):
    prediction = dc.predict(sample)